In [15]:
import optuna
import xgboost as xgb
import pandas as pd
import  numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from xgboost import XGBClassifier 

In [2]:
dff=pd.read_csv(r'D:\360DIGITMG\Machine Learning\DataSet\Admission_Prediction.csv')

In [3]:
dff.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337.0,118.0,4.0,4.5,4.5,9.65,1,0.92
1,2,324.0,107.0,4.0,4.0,4.5,8.87,1,0.76
2,3,NaN,104.0,3.0,3.0,3.5,8.00,1,0.72
3,4,322.0,110.0,3.0,3.5,2.5,8.67,1,0.80
4,5,314.0,103.0,2.0,2.0,3.0,8.21,0,0.65


In [4]:
dff.describe()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,500.000000,485.000000,490.000000,485.000000,500.000000,500.00000,500.000000,500.000000,500.00000
mean,250.500000,316.558763,107.187755,3.121649,3.374000,3.48400,8.576440,0.560000,0.72174
std,144.481833,11.274704,6.112899,1.146160,0.991004,0.92545,0.604813,0.496884,0.14114
min,1.000000,290.000000,92.000000,1.000000,1.000000,1.00000,6.800000,0.000000,0.34000
25%,125.750000,308.000000,103.000000,2.000000,2.500000,3.00000,8.127500,0.000000,0.63000
50%,250.500000,317.000000,107.000000,3.000000,3.500000,3.50000,8.560000,1.000000,0.72000
75%,375.250000,325.000000,112.000000,4.000000,4.000000,4.00000,9.040000,1.000000,0.82000
max,500.000000,340.000000,120.000000,5.000000,5.000000,5.00000,9.920000,1.000000,0.97000


In [5]:
dff['GRE Score'] = dff['GRE Score'].fillna(dff['GRE Score'].mean())
dff['TOEFL Score'] = dff['TOEFL Score'].fillna(dff['TOEFL Score'].mean())
dff['University Rating'] = dff['University Rating'].fillna(dff['University Rating'].mean())

In [11]:
from sklearn.preprocessing import StandardScaler
std_sca = StandardScaler()
x   = std_sca.fit_transform(x)

In [10]:
dff.isna().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

In [8]:
x=dff.drop(columns='Chance of Admit')

In [9]:
y=dff['Chance of Admit']

In [91]:
from sklearn.metrics import mean_squared_error
def objective(trail , data = x , target = y ):
    train_x , test_x ,train_y , test_y = train_test_split(data , target , test_size = .10,random_state = 30  )
    param = {
        
        'lambda':trail.suggest_loguniform('lambda' , 1e-4,10.0),
        'alpha' :trail.suggest_loguniform('alpha' , 1e-4 , 10.0),
        'colsample_bytree' :trail.suggest_categorical('colsample_bytree' , [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
        'subsample' :trail.suggest_categorical('subsample' , [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
        'learning_rate' : trail.suggest_categorical('learning_rate' , [.00001,.0003,.008,.02,.01,1,8]),
        'n_estimator' :30000,
        'max_depth' :trail.suggest_categorical('max_depth', [3,4,5,6,7,8,9,10,11,12]),
        'random_state' :trail.suggest_categorical('random_state' , [10,20,30 ,2000 , 3454,243123]),
        'min_child_weight' :trail.suggest_int('min_child_weight' ,1,200)
    }
    # Fit the model
    optuna_model = XGBRegressor(**param)
    optuna_model.fit(x_train, y_train)

    # Make predictions
    y_pred = optuna_model.predict(x_test)

    # Evaluate predictions
    rmse = mean_squared_error(test_y,y_pred)
    return rmse

In [92]:
find_param=optuna.create_study(direction='minimize')
find_param.optimize(objective,n_trials=10)


[I 2023-12-10 23:42:33,007] A new study created in memory with name: no-name-2eca066a-f591-439c-88a8-221bf0e3ff77
C:\Users\ASUS ZenBook\AppData\Local\Temp\ipykernel_12572\3017389289.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda':trail.suggest_loguniform('lambda' , 1e-4,10.0),
C:\Users\ASUS ZenBook\AppData\Local\Temp\ipykernel_12572\3017389289.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'alpha' :trail.suggest_loguniform('alpha' , 1e-4 , 10.0),
C:\Users\ASUS ZenBook\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:160: UserWarning: [23:42:33] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-

C:\Users\ASUS ZenBook\AppData\Local\Temp\ipykernel_12572\3017389289.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda':trail.suggest_loguniform('lambda' , 1e-4,10.0),
C:\Users\ASUS ZenBook\AppData\Local\Temp\ipykernel_12572\3017389289.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'alpha' :trail.suggest_loguniform('alpha' , 1e-4 , 10.0),
C:\Users\ASUS ZenBook\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:160: UserWarning: [23:42:33] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimator" } are not used.

  warnings.warn(sms

In [94]:
find_param.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_alpha,params_colsample_bytree,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_random_state,params_subsample,state
0,0,0.018205,2023-12-10 23:42:33.015705,2023-12-10 23:42:33.099645,0 days 00:00:00.083940,0.084058,0.7,0.001297,0.00030,11,55,2000,0.6,COMPLETE
1,1,0.010473,2023-12-10 23:42:33.103716,2023-12-10 23:42:33.189913,0 days 00:00:00.086197,4.332589,1.0,0.000296,0.00800,11,38,30,0.9,COMPLETE
2,2,0.018620,2023-12-10 23:42:33.189913,2023-12-10 23:42:33.254711,0 days 00:00:00.064798,0.000356,0.4,0.056260,0.00030,5,111,3454,0.5,COMPLETE
3,3,0.018889,2023-12-10 23:42:33.254711,2023-12-10 23:42:33.323635,0 days 00:00:00.068924,1.813297,0.1,0.000164,0.00001,7,151,10,0.9,COMPLETE
4,4,0.018899,2023-12-10 23:42:33.325650,2023-12-10 23:42:33.379659,0 days 00:00:00.054009,0.008300,0.9,0.249514,0.00001,10,160,3454,0.1,COMPLETE
5,5,0.006877,2023-12-10 23:42:33.383699,2023-12-10 23:42:33.459898,0 days 00:00:00.076199,0.433190,0.2,0.166067,0.02000,4,51,243123,0.4,COMPLETE
6,6,0.018903,2023-12-10 23:42:33.459898,2023-12-10 23:42:33.514443,0 days 00:00:00.054545,0.013155,0.4,0.000317,0.00030,12,100,10,0.3,COMPLETE
7,7,0.018890,2023-12-10 23:42:33.514443,2023-12-10 23:42:33.580145,0 days 00:00:00.065702,0.019054,0.6,2.914662,0.00001,7,89,3454,0.4,COMPLETE
8,8,0.018874,2023-12-10 23:42:33.585200,2023-12-10 23:42:33.659602,0 days 00:00:00.074402,0.001848,0.9,0.032302,0.00001,6,75,243123,0.9,COMPLETE
9,9,0.005062,2023-12-10 23:42:33.666140,2023-12-10 23:42:33.738570,0 days 00:00:00.072430,0.099964,0.8,5.691223,0.02000,9,82,10,0.8,COMPLETE


In [95]:
optuna.visualization.plot_optimization_history(find_param)

In [98]:
optuna.visualization.plot_slice(find_param)

In [97]:
optuna.visualization.plot_contour(find_param , params = ['alpha' , 'lambda'])

In [101]:
best_param= {'lambda': 5.691222842147497, 
  'alpha': 0.09996441259458765, 
  'colsample_bytree': 0.8, 
  'subsample': 0.8, 
  'learning_rate': 0.02, 
  'max_depth': 9, 
  'random_state': 10, 
  'min_child_weight': 82}

In [99]:
old_xgboost=XGBRegressor()
old_xgboost.fit(x,y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [108]:
old_xgboost.score(test_x,test_y)

0.9925790848211412

In [103]:
new_xgboost=XGBRegressor(**best_param)

In [104]:
new_xgboost.fit(x,y)

XGBRegressor(alpha=0.09996441259458765, base_score=None, booster=None,
             callbacks=None, colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, lambda=5.691222842147497,
             learning_rate=0.02, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=9,
             max_leaves=None, min_child_weight=82, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, ...)

In [111]:
new_xgboost.score(test_x,test_y)

0.7472676598257267

In [2]:
import pandas as pd

In [3]:
# classification model

In [16]:
df=pd.read_csv(r'D:\360DIGITMG\Machine Learning\DataSet\winequality-red.csv')

In [33]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [62]:
X=df.drop("quality",axis=1)
y=df['quality']

In [64]:
from  sklearn.preprocessing import LabelEncoder

In [65]:
le=LabelEncoder()
y=le.fit_transform(y)

In [66]:
from sklearn.model_selection import train_test_split

In [67]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20)

In [68]:
xgb=XGBClassifier()
xgb.fit(x_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [69]:
xgb.score(x_test,y_test)

0.6875

In [73]:
def objective_classification(trial,data=X,target=y) :
    train_x , test_x , train_y , test_y= train_test_split(data , target , test_size = .20 ,random_state=10)
    
    param  = {
        
        'verbosity' : 3,
        'objective' : "binary:logistics" , 
        'booster' : trial.suggest_categorical('booster' , ['dart' , 'gbtree','gblinear']),
        'lambda' : trial.suggest_float('lambda' , 1e-4 , 1),
        'alpha' :trial.suggest_float('alpha' , 1e-4 , 1),
        'subsample' : trial.suggest_float('subsample' , .1,.5),
        'colsample_bytree' : trial.suggest_float('colsample_bytree' , .1 ,.5)
        
    }
   
    if param['booster'] in ['gbtree' , 'dart']:
        param['gamma'] :trial.suggest_float('gamma' , 1e-3 , 4 )
        param['eta'] : trial.suggest_float('eta' , .001 ,5 )
        
    xgb_classification = xgboost.XGBClassifier(**param)
    xgb_classification.fit(train_x , train_y,eval_set  = [(test_x, test_y)])
    pred = xgb_classification.predict(test_x)
    accuracy  = xgb_classification.score(test_x , test_y)

    
    
    return accuracy
        

In [74]:
xgb_classification_oputuna = optuna.create_study()

[I 2023-12-11 10:55:36,417] A new study created in memory with name: no-name-129463d4-3dbb-4adb-a056-9754f6a7ee8c


In [75]:
xgb_classification_oputuna.optimize(objective_classification , n_trials = 10 )

[0]	validation_0-mlogloss:1.37445
[1]	validation_0-mlogloss:1.29296
[2]	validation_0-mlogloss:1.25628
[3]	validation_0-mlogloss:1.23476
[4]	validation_0-mlogloss:1.22212
[5]	validation_0-mlogloss:1.21439
[6]	validation_0-mlogloss:1.20930
[7]	validation_0-mlogloss:1.20610
[8]	validation_0-mlogloss:1.20429
[9]	validation_0-mlogloss:1.20360
[10]	validation_0-mlogloss:1.20351
[11]	validation_0-mlogloss:1.20394
[12]	validation_0-mlogloss:1.20469
[13]	validation_0-mlogloss:1.20560
[14]	validation_0-mlogloss:1.20658
[15]	validation_0-mlogloss:1.20753
[16]	validation_0-mlogloss:1.20827
[17]	validation_0-mlogloss:1.20880
[18]	validation_0-mlogloss:1.20920
[19]	validation_0-mlogloss:1.20951
[20]	validation_0-mlogloss:1.20973
[21]	validation_0-mlogloss:1.20990
[22]	validation_0-mlogloss:1.21002
[23]	validation_0-mlogloss:1.21011
[24]	validation_0-mlogloss:1.21018
[25]	validation_0-mlogloss:1.21022
[26]	validation_0-mlogloss:1.21025
[27]	validation_0-mlogloss:1.21027
[28]	validation_0-mlogloss:1.2

C:\Users\ASUS ZenBook\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:160: UserWarning: [10:55:37] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "colsample_bytree", "subsample" } are not used.

  warnings.warn(smsg, UserWarning)


[51]	validation_0-mlogloss:1.21035
[52]	validation_0-mlogloss:1.21035
[53]	validation_0-mlogloss:1.21035
[54]	validation_0-mlogloss:1.21036
[55]	validation_0-mlogloss:1.21036
[56]	validation_0-mlogloss:1.21036
[57]	validation_0-mlogloss:1.21037
[58]	validation_0-mlogloss:1.21037
[59]	validation_0-mlogloss:1.21037
[60]	validation_0-mlogloss:1.21037
[61]	validation_0-mlogloss:1.21038
[62]	validation_0-mlogloss:1.21038
[63]	validation_0-mlogloss:1.21038
[64]	validation_0-mlogloss:1.21038
[65]	validation_0-mlogloss:1.21039
[66]	validation_0-mlogloss:1.21039
[67]	validation_0-mlogloss:1.21039
[68]	validation_0-mlogloss:1.21039
[69]	validation_0-mlogloss:1.21039
[70]	validation_0-mlogloss:1.21040
[71]	validation_0-mlogloss:1.21040
[72]	validation_0-mlogloss:1.21040
[73]	validation_0-mlogloss:1.21040
[74]	validation_0-mlogloss:1.21040
[75]	validation_0-mlogloss:1.21041
[76]	validation_0-mlogloss:1.21041
[77]	validation_0-mlogloss:1.21041
[78]	validation_0-mlogloss:1.21041
[79]	validation_0-ml

[I 2023-12-11 10:55:37,372] Trial 0 finished with value: 0.425 and parameters: {'booster': 'gblinear', 'lambda': 0.17381138712122818, 'alpha': 0.5580885938421619, 'subsample': 0.26742684992125043, 'colsample_bytree': 0.4724951900256583}. Best is trial 0 with value: 0.425.


[10:55:37] ======== Monitor (0): Learner ========
[10:55:37] Configure: 0.001158s, 1 calls @ 1158us

[10:55:37] EvalOneIter: 0.016889s, 100 calls @ 16889us

[10:55:37] GetGradient: 0.027806s, 100 calls @ 27806us

[10:55:37] PredictRaw: 0.012911s, 100 calls @ 12911us

[10:55:37] UpdateOneIter: 0.094576s, 100 calls @ 94576us

[10:55:37] ======== Monitor (0): GBLinear ========
[10:55:37] DoBoost: 0.051449s, 100 calls @ 51449us

[10:55:37] PredictBatch: 0.016662s, 200 calls @ 16662us

[10:55:37] PredictBatchInternal: 0.016334s, 200 calls @ 16334us

[10:55:37] ======== Monitor (0): HostSketchContainer ========
[10:55:37] AllReduce: 0.000219s, 1 calls @ 219us

[10:55:37] MakeCuts: 0.000283s, 1 calls @ 283us

[10:55:37] DEBUG: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\gbm\gbtree.cc:130: Using tree method: 0
[10:55:37] INFO: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboos

...

[I 2023-12-11 10:56:04,204] Trial 8 finished with value: 0.471875 and parameters: {'booster': 'gblinear', 'lambda': 0.49075292246829477, 'alpha': 0.16023201600762255, 'subsample': 0.31761817232783496, 'colsample_bytree': 0.30046319390111287}. Best is trial 0 with value: 0.425.


[0]	validation_0-mlogloss:1.32455
[1]	validation_0-mlogloss:1.27340
[2]	validation_0-mlogloss:1.24808
[3]	validation_0-mlogloss:1.23014
[4]	validation_0-mlogloss:1.21928
[5]	validation_0-mlogloss:1.21204
[6]	validation_0-mlogloss:1.20801
[7]	validation_0-mlogloss:1.20532
[8]	validation_0-mlogloss:1.20354
[9]	validation_0-mlogloss:1.20248
[10]	validation_0-mlogloss:1.20187
[11]	validation_0-mlogloss:1.20173
[12]	validation_0-mlogloss:1.20200
[13]	validation_0-mlogloss:1.20247
[14]	validation_0-mlogloss:1.20305
[15]	validation_0-mlogloss:1.20364
[16]	validation_0-mlogloss:1.20413
[17]	validation_0-mlogloss:1.20463
[18]	validation_0-mlogloss:1.20514
[19]	validation_0-mlogloss:1.20562
[20]	validation_0-mlogloss:1.20606
[21]	validation_0-mlogloss:1.20644
[22]	validation_0-mlogloss:1.20678
[23]	validation_0-mlogloss:1.20707
[24]	validation_0-mlogloss:1.20733
[25]	validation_0-mlogloss:1.20755
[26]	validation_0-mlogloss:1.20775
[27]	validation_0-mlogloss:1.20793
[28]	validation_0-mlogloss:1.2

C:\Users\ASUS ZenBook\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:160: UserWarning: [10:56:04] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "colsample_bytree", "subsample" } are not used.

  warnings.warn(smsg, UserWarning)



[61]	validation_0-mlogloss:1.20972
[62]	validation_0-mlogloss:1.20972
[63]	validation_0-mlogloss:1.20973
[64]	validation_0-mlogloss:1.20973
[65]	validation_0-mlogloss:1.20974
[66]	validation_0-mlogloss:1.20974
[67]	validation_0-mlogloss:1.20975
[68]	validation_0-mlogloss:1.20975
[69]	validation_0-mlogloss:1.20975
[70]	validation_0-mlogloss:1.20975
[71]	validation_0-mlogloss:1.20975
[72]	validation_0-mlogloss:1.20975
[73]	validation_0-mlogloss:1.20975
[74]	validation_0-mlogloss:1.20975
[75]	validation_0-mlogloss:1.20975
[76]	validation_0-mlogloss:1.20975
[77]	validation_0-mlogloss:1.20975
[78]	validation_0-mlogloss:1.20975
[79]	validation_0-mlogloss:1.20975
[80]	validation_0-mlogloss:1.20975
[81]	validation_0-mlogloss:1.20975
[82]	validation_0-mlogloss:1.20975
[83]	validation_0-mlogloss:1.20975
[84]	validation_0-mlogloss:1.20975
[85]	validation_0-mlogloss:1.20975
[86]	validation_0-mlogloss:1.20974
[87]	validation_0-mlogloss:1.20974
[88]	validation_0-mlogloss:1.20974
[89]	validation_0-m

[I 2023-12-11 10:56:04,521] Trial 9 finished with value: 0.43125 and parameters: {'booster': 'gblinear', 'lambda': 0.007229593330676158, 'alpha': 0.3837107194642187, 'subsample': 0.3156047327341532, 'colsample_bytree': 0.3509297622397781}. Best is trial 0 with value: 0.425.


In [76]:
xgb_classification_oputuna.best_params

{'booster': 'gblinear',
 'lambda': 0.17381138712122818,
 'alpha': 0.5580885938421619,
 'subsample': 0.26742684992125043,
 'colsample_bytree': 0.4724951900256583}

In [77]:
param = xgb_classification_oputuna.best_trial.params

In [78]:
xgb_final_clas = xgboost.XGBClassifier(**param)

In [80]:
xgb_final_clas.fit(x_train , y_train)

C:\Users\ASUS ZenBook\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:160: UserWarning: [10:57:30] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "colsample_bytree", "subsample" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(alpha=0.5580885938421619, base_score=None, booster='gblinear',
              callbacks=None, colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.4724951900256583, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, lambda=0.17381138712122818,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, ...)

In [81]:
xgb_final_clas.score(x_test,y_test)

0.446875